NOTE: DO NOT SUBMIT THIS NOTEBOOK FOR YOUR SUBMISSION!!!
PLEASE SUBMIT "A3_submission.py" after you have finished debugging.

Import and setup some auxiliary functions

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from tqdm import tqdm
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.multiprocessing.set_sharing_strategy('file_system')

def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, dataset_name, filename):
    start = timeit.default_timer()
    predicted_test_labels, gt_labels = algorithm(dataset_name)
    if predicted_test_labels is None or gt_labels is None:
      return (0, 0, 0)
    stop = timeit.default_timer()
    run_time = stop - start
    
    np.savetxt(filename, np.asarray(predicted_test_labels))

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
      total += label.size(0)
      correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()   # assuming your model runs on GPU
      
    accuracy = float(correct) / total
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return (correct, accuracy, run_time)

TODO: Implement Logistic Regression here

In [ ]:
def logistic_regression(dataset_name):

    n_epochs = 1
    batch_size_train = 128
    batch_size_test = 1000
    learning_rate = 0.001
    log_interval = 100
    lam = 0.005
    ceLoss = nn.CrossEntropyLoss()

    # predictions and ground truth to be returned
    preds = None
    gts = None

    # Checking GPU availability
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    # Multiple Logistic regression for MNIST
    class MultipleLogisticRegression1(nn.Module):

        def __init__(self):
            super(MultipleLogisticRegression1, self).__init__()
            self.fc = nn.Linear(28*28, 10)

        def forward(self, x):
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return x

    # Multiple Logistic regression for CIFAR10

    class MultipleLogisticRegression2(nn.Module):

        def __init__(self):
            super(MultipleLogisticRegression2, self).__init__()
            self.fc = nn.Linear(3*32*32, 10)

        def forward(self, x):
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return x

    # Following code appears at:  https://lirnli.wordpress.com/2017/09/03/one-hot-encoding-in-pytorch/

    class One_Hot(nn.Module):
        def __init__(self, depth):
            super(One_Hot, self).__init__()
            self.depth = depth
            self.ones = torch.sparse.torch.eye(depth).to(device)

        def forward(self, X_in):
            X_in = X_in.long()
            return self.ones.index_select(0, X_in.data)

        def __repr__(self):
            return self.__class__.__name__ + "({})".format(self.depth)

    if dataset_name == "MNIST":

        MNIST_training = torchvision.datasets.MNIST('/MNIST_dataset/', train=True, download=True,
                                                    transform=torchvision.transforms.Compose([
                                                        torchvision.transforms.ToTensor(),
                                                        torchvision.transforms.Normalize((0.1307,), (0.3081,))]))
        MNIST_test_set = torchvision.datasets.MNIST('/MNIST_dataset/', train=False, download=True,
                                                    transform=torchvision.transforms.Compose([
                                                        torchvision.transforms.ToTensor(),
                                                        torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

        # create a training and a validation set
        MNIST_training_set, MNIST_validation_set = random_split(
            MNIST_training, [48000, 12000])

        # Create data loaders
        train_loader = torch.utils.data.DataLoader(
            MNIST_training_set, batch_size=batch_size_train, shuffle=True)
        validation_loader = torch.utils.data.DataLoader(
            MNIST_validation_set, batch_size=batch_size_train, shuffle=True)
        test_loader = torch.utils.data.DataLoader(
            MNIST_test_set, batch_size=batch_size_test, shuffle=True)

        # setup multi-logistic_model, optimizer, and onehot
        multi_logistic_model = MultipleLogisticRegression1().to(device)
        optimizer = optim.Adam(
            multi_logistic_model.parameters(), lr=learning_rate)
        one_hot = One_Hot(10).to(device)

    elif dataset_name == "CIFAR10":

        # Source: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        CIFAR10_training = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                        download=True, transform=transform)
        CIFAR10_test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                        download=True, transform=transform)

        # create a training and a validation set
        CIFAR10_training_set, CIFAR10_validation_set = random_split(
            CIFAR10_training, [38000, 12000])

        # Create data loaders
        train_loader = torch.utils.data.DataLoader(CIFAR10_training_set,
                                                   batch_size=batch_size_train,
                                                   shuffle=True, num_workers=2)
        validation_loader = torch.utils.data.DataLoader(CIFAR10_validation_set,
                                                        batch_size=batch_size_train,
                                                        shuffle=True, num_workers=2)
        test_loader = torch.utils.data.DataLoader(CIFAR10_test_set,
                                                  batch_size=batch_size_test,
                                                  shuffle=False, num_workers=2)

        multi_logistic_model = MultipleLogisticRegression2().to(device)
        optimizer = optim.Adam(
            multi_logistic_model.parameters(), lr=learning_rate)
        one_hot = One_Hot(10).to(device)

    # Training multi_logistic_model

    def train(epoch):
        multi_logistic_model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = multi_logistic_model(data)
            loss = ceLoss(output, one_hot(target)) + lam * torch.sum(
                torch.sum(torch.pow(list(multi_logistic_model.parameters())[0], 2)), 0)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

    # Validating multi_logistic_model
    def validation():
        multi_logistic_model.eval()
        validation_loss = 0
        correct = 0
        with torch.no_grad():  # notice the use of no_grad
            for data, target in validation_loader:
                data = data.to(device)
                target = target.to(device)
                output = multi_logistic_model(data)
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
                validation_loss += ceLoss(output, one_hot(target)).item() + lam * torch.sum(
                    torch.sum(torch.pow(list(multi_logistic_model.parameters())[0], 2)), 0).item()
        validation_loss = validation_loss / \
            len(validation_loader.dataset) * batch_size_train
        print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            validation_loss, correct, len(validation_loader.dataset), 100. * correct / len(validation_loader.dataset)))

    # Validation benchmark
    validation()

    # Go through epoch range(1, n_epochs + 1)
    for epoch in range(1, n_epochs + 1):
        train(epoch)
        validation()

    multi_logistic_model.eval()
    test_loss = 0
    correct = 0
    count = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = multi_logistic_model(data)
            pred = output.data.max(1, keepdim=True)[1]
            if count == 0:
                preds = pred
                gts = target.data.view_as(pred)
            else:
                preds = torch.cat((preds, pred), 0)
                gts = torch.cat((gts, target.data.view_as(pred)), 0)
            count += 1
            correct += pred.eq(target.data.view_as(pred)).sum()
            test_loss += ceLoss(output, one_hot(target)).item() + lam * torch.sum(
                torch.sum(torch.pow(list(multi_logistic_model.parameters())[0], 2)), 0).item()

    test_loss = test_loss / len(test_loader.dataset) * batch_size_test
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

    preds = preds.to("cpu")
    gts = gts.to("cpu")

    return preds, gts


TODO: Implement Hyper-parameter Tuning here

In [ ]:

def tune_hyper_parameter():

    n_epochs = 1
    batch_size_train = 128
    batch_size_test = 1000
    log_interval = 100
    ceLoss = nn.CrossEntropyLoss()

    # predictions and ground truth to be returned
    preds = None
    gts = None

    # Checking GPU availability
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    # Multiple logistic regression for CIFAR10

    class MultipleLogisticRegression2(nn.Module):

        def __init__(self):
            super(MultipleLogisticRegression2, self).__init__()
            self.fc = nn.Linear(3*32*32, 10)

        def forward(self, x):
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return x

    # Following code appears at:  https://lirnli.wordpress.com/2017/09/03/one-hot-encoding-in-pytorch/
    class One_Hot(nn.Module):
        def __init__(self, depth):
            super(One_Hot, self).__init__()
            self.depth = depth
            self.ones = torch.sparse.torch.eye(depth).to(device)

        def forward(self, X_in):
            X_in = X_in.long()
            return self.ones.index_select(0, X_in.data)

        def __repr__(self):
            return self.__class__.__name__ + "({})".format(self.depth)

    # Source: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    CIFAR10_training = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                    download=True, transform=transform)
    CIFAR10_test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                    download=True, transform=transform)

    # create a training and a validation set
    CIFAR10_training_set, CIFAR10_validation_set = random_split(
        CIFAR10_training, [38000, 12000])

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(CIFAR10_training_set,
                                               batch_size=batch_size_train,
                                               shuffle=True, num_workers=2)
    validation_loader = torch.utils.data.DataLoader(CIFAR10_validation_set,
                                                    batch_size=batch_size_train,
                                                    shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(CIFAR10_test_set,
                                              batch_size=batch_size_test,
                                              shuffle=False, num_workers=2)

    multi_logistic_model = MultipleLogisticRegression2().to(device)
    one_hot = One_Hot(10).to(device)

    # Training multi_logistic_model
    def train(epoch):
        multi_logistic_model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = multi_logistic_model(data)
            loss = ceLoss(output, one_hot(target)) + lam * torch.sum(
                torch.sum(torch.pow(list(multi_logistic_model.parameters())[0], 2)), 0)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

    # Validating multi_logistic_model
    def validation():
        multi_logistic_model.eval()
        validation_loss = 0
        correct = 0
        with torch.no_grad():  # notice the use of no_grad
            for data, target in validation_loader:
                data = data.to(device)
                target = target.to(device)
                output = multi_logistic_model(data)
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
                validation_loss += ceLoss(output, one_hot(target)).item() + lam * torch.sum(
                    torch.sum(torch.pow(list(multi_logistic_model.parameters())[0], 2)), 0).item()
        validation_loss = validation_loss / \
            len(validation_loader.dataset) * batch_size_train
        print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            validation_loss, correct, len(validation_loader.dataset), 100. * correct / len(validation_loader.dataset)))
        return 100. * correct / len(validation_loader.dataset)

    best_accuracy = 0.0
    accuracy = 0.0
    best_lam_adam = 0.0
    best_lam_sgd = 0.0
    lam = 0.0
    learning_rate = 0.0
    best_learning_rate_adam = 0.0
    best_learning_rate_sgd = 0.0

    for i in [0.01, 0.005, 0.001]:
        for j in [0.01, 0.005, 0.001]:
            lam = i
            learning_rate = j
            optimizer = optim.Adam(
                multi_logistic_model.parameters(), lr=learning_rate)
            for epoch in range(1, n_epochs + 1):
                train(epoch)
            accuracy = validation().item()
            print("Validation set accuracy: ", accuracy)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_lam_adam = lam
                best_learning_rate_adam = learning_rate
    print("Best lambda for adam: ", best_lam_adam)
    print("Best learning rate for adam: ", best_learning_rate_adam)

    for i in [0.01, 0.005, 0.001]:
        for j in [0.01, 0.005, 0.001]:
            lam = i
            learning_rate = j
            optimizer = torch.optim.SGD(
                multi_logistic_model.parameters(), lr=learning_rate, momentum=0.9)
            for epoch in range(1, n_epochs + 1):
                train(epoch)
            accuracy = validation().item()
            print("Validation set accuracy: ", accuracy)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_lam_sgd = lam
                best_learning_rate_sgd = learning_rate
    print("Best lambda for adam: ", best_lam_adam)
    print("Best learning rate for adam: ", best_learning_rate_adam)
    print("Best lambda for sgd : ", best_lam_sgd)
    print("Best learning rate for sgd: ", best_learning_rate_sgd)

    return best_lam_adam, best_learning_rate_adam, best_lam_sgd, best_learning_rate_sgd


Main loop. Run time and total score will be shown below.

In [ ]:
def run_on_dataset(dataset_name, filename):
    if dataset_name == "MNIST":
        min_thres = 0.82
        max_thres = 0.92

    elif dataset_name == "CIFAR10":
        min_thres = 0.28
        max_thres = 0.38

    correct_predict, accuracy, run_time = run(logistic_regression, dataset_name, filename)

    score = compute_score(accuracy, min_thres, max_thres)
    result = OrderedDict(correct_predict=correct_predict,
                         accuracy=accuracy, score=score,
                         run_time=run_time)
    return result, score


def main():
    filenames = { "MNIST": "predictions_mnist_YourName_IDNumber.txt", "CIFAR10": "predictions_cifar10_YourName_IDNumber.txt"}
    result_all = OrderedDict()
    score_weights = [0.5, 0.5]
    scores = []
    for dataset_name in ["MNIST","CIFAR10"]:
        result_all[dataset_name], this_score = run_on_dataset(dataset_name, filenames[dataset_name])
        scores.append(this_score)
    total_score = [score * weight for score, weight in zip(scores, score_weights)]
    total_score = np.asarray(total_score).sum().item()
    result_all['total_score'] = total_score
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


main()
start = timeit.default_timer()
tune_hyper_parameter()
stop = timeit.default_timer()
run_time = stop - start
print(run_time)

cuda:0

Validation set: Avg. loss: 2.4080, Accuracy: 1068/12000 (9%)

Train Epoch: 1 [0/48000 (0%)]	Loss: 2.450499
Train Epoch: 1 [12800/48000 (27%)]	Loss: 0.441444
Train Epoch: 1 [25600/48000 (53%)]	Loss: 0.345871
Train Epoch: 1 [38400/48000 (80%)]	Loss: 0.379511

Validation set: Avg. loss: 0.3787, Accuracy: 10863/12000 (91%)


Test set: Avg. loss: 0.3610, Accuracy: 9122/10000 (91%)

Accuracy of the network on the 10000 test images: 91 %
cuda:0
Files already downloaded and verified
Files already downloaded and verified

Validation set: Avg. loss: 2.3468, Accuracy: 1310/12000 (11%)

Train Epoch: 1 [0/38000 (0%)]	Loss: 2.321777
Train Epoch: 1 [12800/38000 (34%)]	Loss: 1.793432
Train Epoch: 1 [25600/38000 (67%)]	Loss: 1.775556

Validation set: Avg. loss: 1.8273, Accuracy: 4680/12000 (39%)


Test set: Avg. loss: 1.8073, Accuracy: 3895/10000 (39%)

Accuracy of the network on the 10000 test images: 38 %

Result:
 OrderedDict([   (   'MNIST',
                    OrderedDict([   ('correct_pre